### **Implementing deep learning techniques for NLP**
- Importing Dependecies:
- Dataset Loading:
- Preprocessing the data:
- Creating vocabulary list using Word2Vec model:
- Averaging Feature Vectors:
- Simple RNN
- LSTM
- GRU


### **Importing Dependecies:**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.pipeline import Pipeline
from bs4 import BeautifulSoup  
import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize, pos_tag
from gensim.models import word2vec
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU,RNN
from keras.preprocessing.text import Tokenizer
from keras.layers.embeddings import Embedding
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Dataset Loading:**

In [3]:
df_final = pd.read_csv('/content/drive/MyDrive/Reviews.csv',low_memory=False)

### **Preprocessing the data:**

Applying various NLP techniques - tokenize and remove all the puncuations and uneseccary jargons.

In [4]:
# Split data into training set and validation
X_train, X_test, y_train, y_test = train_test_split(df_final['Reviews'], df_final['sentiment'], \
                                                    test_size=0.1, random_state=0)

print('Load %d training examples and %d validation examples. \n' %(X_train.shape[0],X_test.shape[0]))
print('Show a review in the training set : \n', X_train.iloc[10])

Load 2459 training examples and 274 validation examples. 

Show a review in the training set : 
 I bought one specifically to play music videos in my car with the Bluetooth sinced to my stereo. It's awesome.


In [5]:
def cleanText(raw_text, remove_stopwords=False, stemming=False, split_text=False, \
             ):
    '''
    Convert a raw review to a cleaned review
    '''
    text = BeautifulSoup(raw_text, 'lxml').get_text()  #remove html
    letters_only = re.sub("[^a-zA-Z]", " ", text)  # remove non-character
    words = letters_only.lower().split() # convert to lower case 
    
    if remove_stopwords: # remove stopword
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
        
    if stemming==True: # stemming
#         stemmer = PorterStemmer()
        stemmer = SnowballStemmer('english') 
        words = [stemmer.stem(w) for w in words]
        
    if split_text==True:  # split text
        return (words)
    
    return( " ".join(words))

In [6]:
# Preprocess text data in training set and validation set
X_train_cleaned = []
X_test_cleaned = []

for d in X_train:
    X_train_cleaned.append(cleanText(d))
print('Show a cleaned review in the training set : \n',  X_train_cleaned[10])
    
for d in X_test:
    X_test_cleaned.append(cleanText(d))

/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "https://www.amazon.com/dp/B01J2G4VBG/refcmcrrypprdttlsol4" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Show a cleaned review in the training set : 
 i bought one specifically to play music videos in my car with the bluetooth sinced to my stereo it s awesome


In [7]:
# Split review text into parsed sentences uisng NLTK's punkt tokenizer

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def parseSent(review, tokenizer, remove_stopwords=False):
    '''
    Parse text into sentences
    '''
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(cleanText(raw_sentence, remove_stopwords, split_text=True))
    return sentences


# Parse each review in the training set into sentences
sentences = []
for review in X_train_cleaned:
    sentences += parseSent(review, tokenizer)
    
print('%d parsed sentence in the training set\n'  %len(sentences))
print('Show a parsed sentence in the training set : \n',  sentences[10])

2459 parsed sentence in the training set

Show a parsed sentence in the training set : 
 ['i', 'bought', 'one', 'specifically', 'to', 'play', 'music', 'videos', 'in', 'my', 'car', 'with', 'the', 'bluetooth', 'sinced', 'to', 'my', 'stereo', 'it', 's', 'awesome']


### **Creating vocabulary list using Word2Vec model:**

Now we have a set of cleaned and parsed sentences from the training data, we can train our own word vector representations by specifying the embedding dimension (= length of feature vector).

In [8]:
# Fit parsed sentences to Word2Vec model 
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

num_features = 300  #embedding dimension                     
min_word_count = 10                
num_workers = 4       
context = 10                                                                                          
downsampling = 1e-3 

print("Training Word2Vec model ...\n")
w2v = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count,\
                 window = context, sample = downsampling)
w2v.init_sims(replace=True)
w2v.save("w2v_300features_10minwordcounts_10context") #save trained word2vec model

print("Number of words in the vocabulary list : %d \n" %len(w2v.wv.index2word)) #4016 
print("Show first 10 words in the vocalbulary list  vocabulary list: \n", w2v.wv.index2word[0:10])

Training Word2Vec model ...

Number of words in the vocabulary list : 776 

Show first 10 words in the vocalbulary list  vocabulary list: 
 ['the', 'it', 'and', 'i', 'to', 'for', 'a', 'is', 'this', 'my']


### **Averaging Feature Vectors:**

Now we have created a vocabulary list of words, with each word having a word representation (ie. feature vector of dim 300).

To find a numerical representation for a review, we run through each word in a review text. For words appear in the vocabulary list, we compute the average feature vectors of all those words. The average feature vector is the numerical represenation of the review.

In [10]:
# Transfrom the training data into feature vectors

def makeFeatureVec(review, model, num_features):
    '''
    Transform a review to a feature vector by averaging feature vectors of words 
    appeared in that review and in the vocabulary list created
    '''
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    index2word_set = set(model.wv.index2word) #index2word is the vocabulary list of the Word2Vec model
    isZeroVec = True
    for word in review:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model[word])
            isZeroVec = False
    if isZeroVec == False:
        featureVec = np.divide(featureVec, nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    '''
    Transform all reviews to feature vectors using makeFeatureVec()
    '''
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        reviewFeatureVecs[counter] = makeFeatureVec(review, model,num_features)
        counter = counter + 1
    return reviewFeatureVecs

In [11]:
# Get feature vectors for training set

X_train_cleaned = []
for review in X_train:
    X_train_cleaned.append(cleanText(review, remove_stopwords=True, split_text=True))
trainVector = getAvgFeatureVecs(X_train_cleaned, w2v, num_features)
print("Training set : %d feature vectors with %d dimensions" %trainVector.shape)


# Get feature vectors for validation set
X_test_cleaned = []
for review in X_test:
    X_test_cleaned.append(cleanText(review, remove_stopwords=True, split_text=True))
testVector = getAvgFeatureVecs(X_test_cleaned, w2v, num_features)
print("Validation set : %d feature vectors with %d dimensions" %testVector.shape)

/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "https://www.amazon.com/dp/B01J2G4VBG/refcmcrrypprdttlsol4" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


Training set : 2459 feature vectors with 300 dimensions
Validation set : 274 feature vectors with 300 dimensions


In [12]:
w2v.wv.most_similar('perfect')

[('has', 0.9998589754104614),
 ('uses', 0.9998309016227722),
 ('item', 0.9998290538787842),
 ('years', 0.9998210072517395),
 ('his', 0.9998196363449097),
 ('who', 0.9998084306716919),
 ('family', 0.9998042583465576),
 ('wanted', 0.9998008608818054),
 ('keeps', 0.9998005628585815),
 ('first', 0.9998002052307129)]

In [13]:
# Load trained Word2Vec model
w2v = Word2Vec.load("w2v_300features_10minwordcounts_10context")


# Get Word2Vec embedding matrix
embedding_matrix = w2v.wv.syn0  # embedding matrix, type = numpy.ndarray 
'''
    The syn0 array essentially holds raw word-vectors. From the perspective of the neural-network used to train 
word-vectors, these vectors are a projection layer that can convert a one-hot encoding of a word into a 
dense embedding-vector of the right dimensionality.
'''
print("Shape of embedding matrix : ", embedding_matrix.shape) #(4016, 300) = (vocabulary size, embedding dimension)
# w2v.wv.syn0[0] #feature vector of the first word in the vocabulary list

Shape of embedding matrix :  (776, 300)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


In [14]:
top_words = embedding_matrix.shape[0] #4016
maxlen = 100 
batch_size = 32
nb_classes = 2
nb_epoch = 3


# Vectorize X_train and X_test to 2D tensor
tokenizer = Tokenizer(nb_words=top_words) #only consider top 20000 words in the corpse
tokenizer.fit_on_texts(X_train)
# tokenizer.word_index #access word-to-index dictionary of trained tokenizer

sequences_train = tokenizer.texts_to_sequences(X_train)
print("Sequences print", sequences_train[1])
sequences_test = tokenizer.texts_to_sequences(X_test)

X_train_seq = sequence.pad_sequences(sequences_train, maxlen=maxlen)
print("Sequences print", X_train_seq[1])
X_test_seq = sequence.pad_sequences(sequences_test, maxlen=maxlen)


# one-hot encoding of y_train and y_test
y_train_seq = np_utils.to_categorical(y_train, nb_classes)

#Converts a class vector (integers) to binary class matrix. E.g. for use with categorical_crossentropy.

print("One Hot", y_train_seq[0])
y_test_seq = np_utils.to_categorical(y_test, nb_classes)

print('X_train shape:', X_train_seq.shape) #(27799, 100)
print('X_test shape:', X_test_seq.shape) #(3089, 100)
print('y_train shape:', y_train_seq.shape) #(27799, 2)
print('y_test shape:', y_test_seq.shape) #(3089, 2)

Sequences print [94, 3, 24, 7, 457, 19, 61, 444]
Sequences print [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0  94   3  24   7 457  19  61 444]
One Hot [0. 1.]
X_train shape: (2459, 100)
X_test shape: (274, 100)
y_train shape: (2459, 2)
y_test shape: (274, 2)


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [15]:
# Construct Word2Vec embedding layer
embedding_layer = Embedding(embedding_matrix.shape[0], #4016
                            embedding_matrix.shape[1], #300
                            weights=[embedding_matrix])

### **Simple RNN:**

  



In [16]:
model = Sequential()
model.add(embedding_layer)
model.add(SimpleRNN(128)) 
model.add(Dense(2,activation="softmax"))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 300)         232800    
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               54912     
                                                                 
 dense (Dense)               (None, 2)                 258       
                                                                 
Total params: 287,970
Trainable params: 287,970
Non-trainable params: 0
_________________________________________________________________


In [21]:
histRNN = model.fit(X_train_seq,y_train_seq,epochs = 10, verbose=1)

Epoch 1/10
77/77 [==============================] - 6s 63ms/step - loss: 0.2162 - accuracy: 0.9479
Epoch 2/10
77/77 [==============================] - 5s 63ms/step - loss: 0.1876 - accuracy: 0.9479
Epoch 3/10
77/77 [==============================] - 5s 62ms/step - loss: 0.1409 - accuracy: 0.9500
Epoch 4/10
77/77 [==============================] - 6s 72ms/step - loss: 0.0726 - accuracy: 0.9744
Epoch 5/10
77/77 [==============================] - 5s 62ms/step - loss: 0.0577 - accuracy: 0.9817
Epoch 6/10
77/77 [==============================] - 5s 62ms/step - loss: 0.0171 - accuracy: 0.9967
Epoch 7/10
77/77 [==============================] - 5s 63ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 8/10
77/77 [==============================] - 5s 62ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 9/10
77/77 [==============================] - 5s 63ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 10/10
77/77 [==============================] - 5s 63ms/step - loss: 7.6934e-04 - accuracy: 1.0000


### **LSTM:**

In [22]:
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(64)) 
model.add(Dense(2,activation="softmax"))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 300)         232800    
                                                                 
 lstm (LSTM)                 (None, 64)                93440     
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 326,370
Trainable params: 326,370
Non-trainable params: 0
_________________________________________________________________


In [23]:
histLSTM = model.fit(X_train_seq,y_train_seq,epochs = 10, verbose=1)

Epoch 1/10
77/77 [==============================] - 18s 193ms/step - loss: 0.2329 - accuracy: 0.9402
Epoch 2/10
77/77 [==============================] - 15s 192ms/step - loss: 0.1650 - accuracy: 0.9479
Epoch 3/10
77/77 [==============================] - 15s 193ms/step - loss: 0.1113 - accuracy: 0.9532
Epoch 4/10
77/77 [==============================] - 15s 193ms/step - loss: 0.0682 - accuracy: 0.9748
Epoch 5/10
77/77 [==============================] - 15s 200ms/step - loss: 0.0421 - accuracy: 0.9858
Epoch 6/10
77/77 [==============================] - 15s 193ms/step - loss: 0.0283 - accuracy: 0.9927
Epoch 7/10
77/77 [==============================] - 15s 193ms/step - loss: 0.0225 - accuracy: 0.9943
Epoch 8/10
77/77 [==============================] - 15s 196ms/step - loss: 0.0123 - accuracy: 0.9963
Epoch 9/10
77/77 [==============================] - 15s 198ms/step - loss: 0.0090 - accuracy: 0.9984
Epoch 10/10
77/77 [==============================] - 15s 194ms/step - loss: 0.0041 - accura

### **Validation of trained model with test data**

In [24]:
# model.predict(X_test_seq)
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
prediction = model.predict(X_test_seq)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test_seq))
print('Confusion matrix:')
confusion_matrix(y_test_seq.argmax(axis=1),y_pred.argmax(axis=1))

Accuracy of the model :  0.9452554744525548
Confusion matrix:


array([[  5,  12],
       [  3, 254]])

### **GRU:**

In [25]:
model = Sequential()
model.add(embedding_layer)
model.add(GRU(64)) 
model.add(Dense(2,activation="softmax"))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 300)         232800    
                                                                 
 gru (GRU)                   (None, 64)                70080     
                                                                 
 dense_2 (Dense)             (None, 2)                 130       
                                                                 
Total params: 303,010
Trainable params: 303,010
Non-trainable params: 0
_________________________________________________________________


In [ ]:
histGRU = model.fit(X_train_seq,y_train_seq,epochs = 10, verbose=1)

Epoch 1/10
77/77 [==============================] - 11s 105ms/step - loss: 0.1821 - accuracy: 0.9475
Epoch 2/10
77/77 [==============================] - 8s 101ms/step - loss: 0.0729 - accuracy: 0.9683
Epoch 3/10
77/77 [==============================] - 8s 103ms/step - loss: 0.0223 - accuracy: 0.9947
Epoch 4/10
77/77 [==============================] - 8s 102ms/step - loss: 0.0071 - accuracy: 0.9996
Epoch 5/10
77/77 [==============================] - 8s 101ms/step - loss: 0.0073 - accuracy: 0.9984
Epoch 6/10
77/77 [==============================] - 8s 100ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 7/10
77/77 [==============================] - 8s 102ms/step - loss: 0.0018 - accuracy: 0.9996
Epoch 8/10
77/77 [==============================] - 8s 101ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 9/10
77/77 [==============================] - 8s 101ms/step - loss: 4.1428e-04 - accuracy: 1.0000
Epoch 10/10
77/77 [==============================] - 8s 101ms/step - loss: 2.8629e-04 - accurac

In [26]:
# model.predict(X_test_seq)
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
prediction = model.predict(X_test_seq)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test_seq))
print('Confusion matrix:')
confusion_matrix(y_test_seq.argmax(axis=1),y_pred.argmax(axis=1))

Accuracy of the model :  0.24452554744525548
Confusion matrix:


array([[ 11,   6],
       [201,  56]])